In [ ]:
import datalab.bigquery as bq
import google.datalab.storage as storage
from datalab.context import Context
import pandas as pd
from pandas.io import gbq
import math

projectId = Context.default().project_id

In [ ]:
def create_content_drive(text):
  print 'Generating the result...'
  bucket_name = 'ciandt_projectoctopus_2018_felipegc'
  bucket = storage.Bucket(bucket_name)
  result_object = bucket.object('predictions.csv')
  result_object.write_stream(text, 'text/plain')


In [ ]:
sql_percent_record = """
SELECT 
  CASE
    WHEN team = 'South Korea' THEN 'Korea Republic'
    ELSE team
  END AS team,
  SUM(cnt_performance_match) as num_matches,
  SUM(cnt_performance_match) * 3 AS max_points,
  ROUND(SUM(percent_record) / (SUM(cnt_performance_match) * 3), 2) AS percent_record,
  SUM(scores) AS total_scores
FROM (
  SELECT
    team,
    what_happened,
    cnt_performance_match,
    CASE
      WHEN what_happened = 'lost' THEN cnt_performance_match * 0
      WHEN what_happened = 'tied' THEN cnt_performance_match * 1
      WHEN what_happened = 'won' THEN cnt_performance_match * 3
    END AS percent_record,
    scores
  FROM (
    SELECT
      team,
      what_happened,
      COUNT(*) AS cnt_performance_match,
      SUM(scores) AS scores
    FROM (
      SELECT
        *
      FROM (
        SELECT
          year,
          home AS team,
          home_score AS scores,
          CASE
            WHEN home_score > away_score THEN 'won'
            WHEN home_score < away_score THEN 'lost'
            WHEN home_score = away_score THEN 'tied'
          END AS what_happened
        FROM
          [paul_the_octopus_dataset.matches_history] ) home,
        (
        SELECT
          year,
          away AS team,
          away_score AS scores,
          CASE
            WHEN away_score > home_score THEN 'won'
            WHEN away_score < home_score THEN 'lost'
            WHEN home_score = away_score THEN 'tied'
          END AS what_happened
        FROM
          [paul_the_octopus_dataset.matches_history] ) away
      ORDER BY
        year ASC)
    GROUP BY
      1,
      2) )
GROUP BY
  1
"""
print 'Running query...'
percent_record = gbq.read_gbq(sql_percent_record, project_id=projectId)
percent_record[:5]

In [ ]:
sql_rank = """
SELECT
  rank,
  team,
  total_points,
  INTEGER(rank / 5) + 1 AS rank_group
FROM
  [paul_the_octopus_dataset.fifa_rank]
"""

print 'Running query...'
rank = gbq.read_gbq(sql_rank, project_id=projectId)
rank[:15]

In [ ]:
team_status = \
  rank.merge(percent_record, left_on='team', right_on='team', how='left').sort_values(by='rank', ascending=True).fillna(0)

team_status[:50]

In [ ]:
def calculate_weights(team_1, team_2):
  #normalizing and applying weights if necessary
  weight_percent_record = 1000 * 3 
  weight_rank = 1 * 2
  weight_scores = 10 * 1
  
  team_1_percent_record = team_1['percent_record'].values[0] * weight_percent_record
  team_2_percent_record = team_2['percent_record'].values[0] * weight_percent_record
  
  team_1_rank = team_1['total_points'].values[0] * weight_rank
  team_2_rank = team_2['total_points'].values[0] * weight_rank
  
  team_1_scores = team_1['total_scores'].values[0] * weight_scores
  team_2_scores = team_2['total_scores'].values[0] * weight_scores
  
  team_1_weight = int(team_1_percent_record + team_1_rank + team_1_scores)
  team_2_weight = int(team_2_percent_record + team_2_rank + team_2_scores)
  
  return team_1_weight, team_2_weight
  

In [ ]:
def calculate_winner(team_1, team_2):
  winner = ''
  diff_weight = 150
  
  team_1_weight, team_2_weight = calculate_weights(team_1, team_2)
  
  print team_1_weight
  print team_2_weight

  if team_1_weight - team_2_weight > diff_weight:
    winner = 'team_1'
  elif team_2_weight - team_1_weight > diff_weight:
    winner = 'team_2'
  else:
    winner = 'no_one'
    
  print 'winner calculated is {}'.format(winner)
  
  return winner, team_1_weight, team_2_weight

In [ ]:
def calculate_score(team_1, team_2, team_1_weight, team_2_weight, winner):
  
  def calculate_avg_score(team):
      avg_scores_per_match = int(team['total_scores'].values[0] / team['num_matches'].values[0]) \
                           if team['num_matches'].values[0] > 0 else 0
      return avg_scores_per_match
  
  def play_match(winner, looser, weight_winner, weight_looser, tied):
    winner_score = 0
    looser_score = 0
    
    if tied:
      tied_score = calculate_avg_score(winner) if weight_winner > weight_looser else calculate_avg_score(looser)
      print 'tied_score -> {}'.format(tied_score)

      return tied_score, tied_score
      
    winner_score = math.ceil(weight_winner / (weight_looser * 1.0))
    
    looser_avg_scores = calculate_avg_score(looser) 
    looser_score = winner_score - 1 if looser_avg_scores >= winner_score else looser_avg_scores

    print 'winner_score -> {}'.format(winner_score)
    print 'looser_score -> {}'.format(looser_score)
    
    return winner_score, looser_score
    
  if winner == 'team_1':
    score_left, score_right = play_match(team_1, team_2, team_1_weight, team_2_weight, False)
    return team_1['team'].values[0], score_left, score_right, team_2['team'].values[0]
  elif winner == 'team_2':
    #swap the score position since team_2 belongs to the right side(away) of the match
    score_right, score_left = play_match(team_2, team_1, team_2_weight, team_1_weight, False)
    return team_1['team'].values[0], score_left, score_right, team_2['team'].values[0]
  else:
    tied_score, tied_score = play_match(team_1, team_2, team_1_weight, team_2_weight, True)
    return team_1['team'].values[0], tied_score, tied_score, team_2['team'].values[0]
  

In [ ]:
def predict_match(team_1, team_2):
  winner, team_1_weight, team_2_weight = calculate_winner(team_1, team_2)
  return calculate_score(team_1, team_2, team_1_weight, team_2_weight, winner)
  

In [ ]:
sql_matches = """
SELECT
  date,
  RTRIM(LTRIM(home)) AS team_1,
  RTRIM(LTRIM(away)) AS team_2,
FROM
  [paul_the_octopus_dataset.matches]
"""

matches = gbq.read_gbq(sql_matches, project_id=projectId)
matches[:5]


In [ ]:
def predict_matches():
  print 'predicting matches...'
  out = 'home,home_score,away_score,away'
  
  for index, row in matches.iterrows():
    print 'predicting match between {} X {}'.format(row['team_1'], row['team_2'])
    
    team_1_data = team_status.loc[team_status['team'] == row['team_1']]
    team_2_data = team_status.loc[team_status['team'] == row['team_2']]
    
    team_1, score_1, score_2, team_2 = predict_match(team_1_data, team_2_data)
    out += '\r\n{},{},{},{}'.format(team_1, score_1, score_2, team_2)
  
  print out
  
  return out
    

In [ ]:
# team_1_data = team_status.loc[team_status['team'] == 'Brazil']
# team_2_data = team_status.loc[team_status['team'] == 'Germany']
# predict_match(team_1_data, team_2_data)

create_content_drive(predict_matches())